### Load Packages

In [2]:
import re
import pandas as pd

### Join Dataframes

#### Heights and Weights

Source: http://stats.nba.com/league/player/#!/bio/

In [3]:
height_weight = pd.read_csv('data/height_weight.csv')
height_weight.tail()

,Player,Age,Height,Weight
456,Wilson Chandler,29,6-8,225
457,Yogi Ferrell,23,6-1,180
458,Zach LaVine,21,6-5,185
459,Zach Randolph,35,6-9,260
460,Zaza Pachulia,33,6-11,275


In [4]:
height_weight.shape

(461, 4)

#### Positions

Source: http://www.basketball-reference.com/leagues/NBA_2017_totals.html

In [5]:
pos_age = pd.read_csv('data/position_age.csv')
pos_age = pos_age.drop_duplicates()
pos_age.tail()

,Player,Pos
476,Cody Zeller,PF
477,Tyler Zeller,C
478,Stephen Zimmerman,C
479,Paul Zipser,SF
480,Ivica Zubac,C


In [6]:
pos_age.shape

(461, 2)

#### Save to CSV as tmp_df.csv

In [7]:
pos_age.to_csv('data/tmp_df.csv')

#### Merge Dataframes

In [8]:
df = pd.merge(height_weight, pos_age, how='inner', on='Player')
df.tail()

,Player,Age,Height,Weight,Pos
435,Wilson Chandler,29,6-8,225,SF
436,Yogi Ferrell,23,6-1,180,PG
437,Zach LaVine,21,6-5,185,SG
438,Zach Randolph,35,6-9,260,PF
439,Zaza Pachulia,33,6-11,275,C


In [9]:
cleaned_player = []

for player in df['Player']:
    player = player.lower()
    player = player.strip()
    player = (player.replace(',','').replace('.','').replace("'",""))
    cleaned_player.append(player)
    
df['cleaned_player'] = cleaned_player

In [10]:
df = df.drop('Player', axis=1)

#### Save to CSV

In [11]:
df.to_csv('data/first_merge.csv')

#### Web Scraped Data

In [12]:
scraped_data = pd.read_csv('data/web_scraped_stats.csv')
scraped_data.head()

,Player,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DPM,BPM,VORP
0,Alex Abrines,42,0,14.0,1.8,4.5,0.396,1.2,3.2,0.378,...,10.2,16.4,0.6,0.5,1.1,0.088,0,-2.6,-3.2,-0.2
1,Quincy Acy,237,51,14.9,1.7,3.5,0.495,0.2,0.7,0.359,...,13.0,13.8,4.4,2.8,7.3,0.099,0,0.0,-1.5,0.5
2,Steven Adams,282,218,23.2,2.9,5.2,0.568,0.0,0.0,0.000,...,16.8,13.9,9.3,8.7,18.0,0.132,0,1.6,0.9,4.8
3,Arron Afflalo,693,519,28.4,4.1,9.2,0.450,1.2,3.0,0.385,...,10.4,18.0,23.4,9.3,32.7,0.080,0,-1.3,-1.0,5.2
4,Alexis Ajinca,275,71,13.2,2.2,4.3,0.500,0.0,0.1,0.293,...,15.1,19.4,2.7,4.2,6.8,0.091,0,0.4,-2.5,-0.4


In [13]:
scraped_data.shape

(454, 65)

In [14]:
cleaned_player = []

for player in scraped_data['Player']:
    player = player.lower()
    player = player.strip()
    player = (player.replace(',','').replace('.','').replace("'",""))
    cleaned_player.append(player)
    
scraped_data['cleaned_player'] = cleaned_player

In [15]:
data = pd.merge(df, scraped_data, how='right', on='cleaned_player')
data.rename(columns={'PF':'Fouls'}, inplace=True)
data.head()

,Age,Height,Weight,Pos,cleaned_player,Player,G,GS,MP,FG,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DPM,BPM,VORP
0,32.0,6-1,161.0,PG,aaron brooks,Aaron Brooks,593,182,21.9,3.7,...,14.4,23.3,10.5,8.6,19.1,0.070,0,-2.4,-1.7,1.1
1,21.0,6-9,220.0,SF,aaron gordon,Aaron Gordon,179,91,23.2,3.4,...,9.6,17.8,4.1,4.0,8.1,0.094,0,0.4,-0.2,1.8
2,22.0,6-6,210.0,SG,aaron harrison,Aaron Harrison,26,0,4.2,0.2,...,12.1,13.6,-0.2,0.2,-0.1,-0.033,0,-0.2,-6.4,-0.1
3,25.0,6-10,237.0,PF,adreian payne,Adreian Payne,96,24,13.7,1.7,...,16.4,17.8,-1.5,0.7,-0.8,-0.030,0,-0.8,-5.8,-1.3
4,30.0,6-10,245.0,C,al horford,Al Horford,618,614,33.4,6.1,...,11.1,19.6,38.2,29.6,67.9,0.158,0,2.2,3.0,26.2


In [16]:
data = data.drop('Player', axis=1)

### Clean the Data

#### Clean the `Height` column

In [17]:
def inches(string):
    try: 
        result = re.match("(\d)-(\d+)", string)
        feet = int(result.group(1))
        if result.group(2) == "":
            inches = 0
        else:
            inches = int(result.group(2))
    except:
        return None  
    return feet * 12 + inches

In [18]:
data['Height'] = data['Height'].apply(inches)
data.head()

,Age,Height,Weight,Pos,cleaned_player,G,GS,MP,FG,FGA,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DPM,BPM,VORP
0,32.0,73.0,161.0,PG,aaron brooks,593,182,21.9,3.7,9.0,...,14.4,23.3,10.5,8.6,19.1,0.070,0,-2.4,-1.7,1.1
1,21.0,81.0,220.0,SF,aaron gordon,179,91,23.2,3.4,7.4,...,9.6,17.8,4.1,4.0,8.1,0.094,0,0.4,-0.2,1.8
2,22.0,78.0,210.0,SG,aaron harrison,26,0,4.2,0.2,0.9,...,12.1,13.6,-0.2,0.2,-0.1,-0.033,0,-0.2,-6.4,-0.1
3,25.0,82.0,237.0,PF,adreian payne,96,24,13.7,1.7,4.2,...,16.4,17.8,-1.5,0.7,-0.8,-0.030,0,-0.8,-5.8,-1.3
4,30.0,82.0,245.0,C,al horford,618,614,33.4,6.1,11.6,...,11.1,19.6,38.2,29.6,67.9,0.158,0,2.2,3.0,26.2


In [19]:
# Get Dummies for Player Positions
# pos_df = pd.get_dummies(data['Pos'])
# df_new = pd.concat([data, pos_df], axis=1)

In [20]:
data.columns

Index([u'Age', u'Height', u'Weight', u'Pos', u'cleaned_player', u'G', u'GS',
       u'MP', u'FG', u'FGA', u'FG%', u'3P', u'3PA', u'3P%', u'2P', u'2PA',
       u'2P%', u'eFG%', u'FT', u'FTA', u'FT%', u'ORB', u'DRB', u'TRB', u'AST',
       u'STL', u'BLK', u'TOV', u'Fouls', u'PTS', u'AVG_DIST_FGA', u'%FGA_2P',
       u'%FGA_0-3ft', u'%FGA_3-10ft', u'%FGA_10-16ft', u'%FGA_16ft<3',
       u'%FGA_3P', u'0-3_FG%', u'3-10_FG%', u'10-16_FG%', u'16<3_FG%',
       u'%ASTd_2P', u'%FGA_DUNK', u'DUNKS', u'%ASTd_3P', u'%_CORNER3PA',
       u'3P%_CORNER3', u'HEAVE_ATT', u'HEAVE_MD', u'PER', u'TS%', u'3PAr',
       u'FTr', u'ORB%', u'DRB%', u'TRB%', u'AST%', u'STL%', u'BLK%', u'TOV%',
       u'USG%', u'OWS', u'DWS', u'WS', u'WS/48', u'OBPM', u'DPM', u'BPM',
       u'VORP'],
      dtype='object')

In [21]:
df_new = data.drop(['G', 'GS', 'DUNKS', 'HEAVE_ATT', 'HEAVE_MD', 'WS', 'OWS', 'DWS', 'WS/48', 'VORP'], axis=1)
df_new.head()

,Age,Height,Weight,Pos,cleaned_player,MP,FG,FGA,FG%,3P,...,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OBPM,DPM,BPM
0,32.0,73.0,161.0,PG,aaron brooks,21.9,3.7,9.0,0.413,1.3,...,6.9,4.5,23.8,1.4,0.5,14.4,23.3,0,-2.4,-1.7
1,21.0,81.0,220.0,SF,aaron gordon,23.2,3.4,7.4,0.451,0.6,...,17.8,12.3,9.8,1.4,2.0,9.6,17.8,0,0.4,-0.2
2,22.0,78.0,210.0,SG,aaron harrison,4.2,0.2,0.9,0.217,0.1,...,14.0,8.9,6.0,2.7,0.0,12.1,13.6,0,-0.2,-6.4
3,25.0,82.0,237.0,PF,adreian payne,13.7,1.7,4.2,0.399,0.1,...,19.4,12.8,7.4,1.5,1.5,16.4,17.8,0,-0.8,-5.8
4,30.0,82.0,245.0,C,al horford,33.4,6.1,11.6,0.529,0.3,...,22.1,15.1,14.7,1.3,2.9,11.1,19.6,0,2.2,3.0


### Save to CSV

In [22]:
df_new.to_csv('data/cleaned_player_stats_no_dummies.csv')